<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

# Изучение БД сервиса для чтения книг

ЦЕЛЬ:  Анализ базы данных сервиса для чтения книг, выявление основных показателей, формулировка ценностного предложения для нового продукта.

Залачи исследования:
- Посчитать, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитать количество обзоров и среднюю оценку;
- Определить издательство, которое выпустило наибольшее число книг толще 50 страниц;
- Определить автора с самой высокой средней оценкой книг;
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [1]:
# импортируем библиотеки
import pandas as pd
from sqlalchemy import text, create_engine

# устанавливаем параметры
db_config = {'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db'} # название базы данных
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)

# сохраняем коннектор
engine = create_engine(connection_string, connect_args={'sslmode':'require'})

In [2]:
mylist = ['books', 'authors', 'publishers', 'ratings', 'reviews']
for name in mylist:
    query = '''SELECT * FROM {} LIMIT 5'''.format(name)
    con=engine.connect()
    print('Первые 5 строк из таблицы', name)
    display(pd.io.sql.read_sql(sql=text(query), con = con))
    

Первые 5 строк из таблицы books


,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


Первые 5 строк из таблицы authors


,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


Первые 5 строк из таблицы publishers


,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


Первые 5 строк из таблицы ratings


,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


Первые 5 строк из таблицы reviews


,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


Вывод: 

Описание данных

**Таблица `books`**

Содержит данные о книгах:

- `book_id` — идентификатор книги;
- `author_id` — идентификатор автора;
- `title` — название книги;
- `num_pages` — количество страниц;
- `publication_date` — дата публикации книги;
- `publisher_id` — идентификатор издателя.

**Таблица `authors`**

Содержит данные об авторах:

- `author_id` — идентификатор автора;
- `author` — имя автора.

**Таблица `publishers`**

Содержит данные об издательствах:

- `publisher_id` — идентификатор издательства;
- `publisher` — название издательства;

**Таблица `ratings`**

Содержит данные о пользовательских оценках книг:

- `rating_id` — идентификатор оценки;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, оставившего оценку;
- `rating` — оценка книги.

**Таблица `reviews`**

Содержит данные о пользовательских обзорах на книги:

- `review_id` — идентификатор обзора;
- `book_id` — идентификатор книги;
- `username` — имя пользователя, написавшего обзор;
- `text` — текст обзора.

In [3]:
query = '''
            SELECT COUNT(title)
            FROM books
            WHERE publication_date>'2000-01-01' 
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,count
0,819


Вывод: 819 книг вышло после 1 января 2000 года

In [27]:
query = '''
            WITH t_1 AS (SELECT book_id, COUNT(review_id) AS count
                         FROM reviews
                         GROUP BY book_id),
                         
                 t_2 AS (SELECT book_id, AVG(rating) AS avg
                         FROM ratings
                         GROUP BY book_id)
            
            SELECT b.title, t_1.count, t_2.avg
            FROM books AS b
            LEFT JOIN t_1 AS t_1 ON b.book_id = t_1.book_id
            LEFT JOIN t_2 AS t_2 ON b.book_id = t_2.book_id
            ORDER BY t_2.avg DESC
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,title,count,avg
0,A Dirty Job (Grim Reaper #1),4.0,5.00
1,Tai-Pan (Asian Saga #2),2.0,5.00
2,School's Out—Forever (Maximum Ride #2),3.0,5.00
3,Piercing the Darkness (Darkness #2),2.0,5.00
4,The War of Art: Break Through the Blocks & Win...,2.0,5.00
...,...,...,...
995,The World Is Flat: A Brief History of the Twen...,3.0,2.25
996,His Excellency: George Washington,2.0,2.00
997,Junky,2.0,2.00
998,Drowning Ruth,3.0,2.00


Вывод: Для каждой книги посчитано количество обзоров и средняя оценка. Самая большая оценка - 5, а самая маленькая - 1.5

In [5]:
query = '''
            SELECT publisher, (SELECT COUNT(title)
                                    FROM books
                                    WHERE num_pages>50
                                    GROUP BY publisher_id
                                    ORDER BY COUNT(title) DESC
                                    LIMIT 1)
            FROM publishers
            WHERE publisher_id IN (SELECT publisher_id
                                    FROM books
                                    WHERE num_pages>50
                                    GROUP BY publisher_id
                                    ORDER BY COUNT(title) DESC
                                    LIMIT 1)
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,publisher,count
0,Penguin Books,42


Вывод: Penguin Books - издательство, которое выпустило наибольшее число книг толще 50 страниц - 42

In [6]:
query = '''
            SELECT author, AVG(rating)
                FROM authors AS a
                INNER JOIN books AS b ON a.author_id= b.author_id
                INNER JOIN ratings AS r ON b.book_id= r.book_id
                WHERE b.book_id IN (SELECT book_id
                                    FROM ratings
                                    GROUP BY book_id
                                    HAVING COUNT(rating)>50)
                GROUP BY author
                ORDER BY AVG(rating) DESC
                LIMIT 1
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,author,avg
0,J.K. Rowling/Mary GrandPré,4.287097


Вывод: J.K. Rowling/Mary GrandPré - автор с самой высокой средней оценкой книг 4,3

In [7]:
query = '''
        SELECT AVG(count)
        FROM (SELECT COUNT(text) AS count
                FROM reviews
                WHERE username IN (SELECT username
                                    FROM ratings
                                    GROUP BY username
                                    HAVING COUNT(rating)>48)
                GROUP BY username) AS t                   
        '''

con=engine.connect()

pd.io.sql.read_sql(sql=text(query), con = con)

,avg
0,24.0


Вывод: среднее количество обзоров от пользователей, которые поставили больше 48 оценок = 24

Общий вывод:  

Цель выполнена: Анализ базы данных сервиса для чтения книг произведен, выявлены основные показатели, сформулировано ценностное предложение для нового продукта.

Залачи исследования выполнены:
- Посчитано, сколько книг вышло после 1 января 2000 года;
    * 819 книг вышло после 1 января 2000 года
- Для каждой книги посчитано количество обзоров и среднюю оценку;
    * Самая большая оценка - 5, а самая маленькая - 1.5
- Определено издательство, которое выпустило наибольшее число книг толще 50 страниц;
    * Penguin Books - 42 книги
- Определен автор с самой высокой средней оценкой книг;
    * J.K. Rowling/Mary GrandPré - 4.3
- Посчитайть среднее количество обзоров от пользователей, которые поставили больше 48 оценок
    * Среднее равно = 24